In [1]:
import pandas as pd

In [2]:
! git clone https://github.com/andrealodi/ORIE-5213-Spring2023.git

Cloning into 'ORIE-5213-Spring2023'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 553 (delta 2), reused 0 (delta 0), pack-reused 533
Receiving objects: 100% (553/553), 3.15 MiB | 21.23 MiB/s, done.
Resolving deltas: 100% (505/505), done.


In [3]:
initial_inven= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/Initial_Inven.json")

In [4]:
initial_inven

,0
0,7
1,0
2,1
3,9
4,0
5,9
6,5
7,18
8,3
9,5


In [5]:
# number of bikes initially at stand 20
initial_inven.iloc[20]

0    13
Name: 20, dtype: int64

In [6]:
simu0= pd.read_json("ORIE-5213-Spring2023/Bike sharing/Dataset/simu0_0.json")

In [7]:
simu0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       1458 non-null   int64
 1   1       1458 non-null   int64
 2   2       1458 non-null   int64
 3   3       1458 non-null   int64
dtypes: int64(4)
memory usage: 45.7 KB


In [8]:
simu0.describe()

,0,1,2,3
count,1458.000000,1458.000000,1458.000000,1458.000000
mean,834.290123,14.154321,846.270919,13.951989
std,293.686675,9.045841,295.914786,9.105823
min,4.000000,0.000000,1.000000,0.000000
25%,553.750000,6.000000,565.250000,5.000000
50%,903.500000,15.000000,911.500000,14.000000
75%,1060.750000,22.000000,1077.000000,22.000000
max,1439.000000,29.000000,1429.000000,29.000000


In [9]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,3,9
1454,1417,17,1428,3
1455,1432,0,14,8
1456,1439,9,21,29


In [10]:
# Adjust column 2 based on values in column 0
simu0.loc[simu0[0] > simu0[2], 2] += 1440


In [11]:
simu0

,0,1,2,3
0,4,1,22,11
1,15,1,37,18
2,19,12,45,8
3,48,11,72,5
4,50,22,73,27
...,...,...,...,...
1453,1414,27,1443,9
1454,1417,17,1428,3
1455,1432,0,1454,8
1456,1439,9,1461,29


In [12]:
import pandas as pd

# Define the time ranges
time_range = [(i, i + 15) for i in range(0, 1440, 15)]

# Create the function to assign the time range
def assign_time_range(value):
    for start, end in time_range:
        if start <= value < end:
            return f"{start}-{end}"
    return "Unknown"

# Create the 'departure_time' column
simu0['departure_time'] = simu0[0].apply(assign_time_range)
simu0['arrival_time'] = simu0[2].apply(assign_time_range)

In [13]:
simu0.groupby([1,'departure_time'])[0].count().reset_index()


,1,departure_time,0
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [14]:
# import pandas as pd

# # Define the time ranges
# time_range = list(i for i in range(0, 1455, 15))

# # Create the 'departure_time' column
# simu0['departure_time'] = pd.cut(simu0[0], bins=time_range, labels=time_range[:-1])
# simu0['arrival_time'] = pd.cut(simu0[2], bins=time_range, labels=time_range[:-1])

# # Convert the 'departure_time' column to integer type
# simu0['departure_time'] = simu0['departure_time'].astype(int)
# simu0['arrival_time'] = simu0['arrival_time'].astype(int)
# simu0

In [15]:
station0 = simu0[(simu0[1]==0)]
station0[station0['departure_time']=='1020-1035']


,0,1,2,3,departure_time,arrival_time
955,1021,0,1045,5,1020-1035,1035-1050
964,1024,0,1052,27,1020-1035,1050-1065
993,1032,0,1048,5,1020-1035,1035-1050
998,1033,0,1046,15,1020-1035,1035-1050
999,1033,0,1060,10,1020-1035,1050-1065
1004,1034,0,1055,29,1020-1035,1050-1065


In [16]:
df_rentals = simu0.groupby([1, 'departure_time'])[0].count().reset_index()
df_rentals = df_rentals.rename(columns={0:"rentals", 1: "station_id"})
df_rentals

,station_id,departure_time,rentals
0,0,1005-1020,5
1,0,1020-1035,6
2,0,1035-1050,3
3,0,1050-1065,3
4,0,1065-1080,2
...,...,...,...
948,29,855-870,1
949,29,915-930,1
950,29,930-945,2
951,29,960-975,3


In [17]:
df_returns = simu0.groupby([3, 'arrival_time'])[2].count().reset_index()
df_returns = df_returns.rename(columns={2:"returns", 3: "station_id"})
df_returns

,station_id,arrival_time,returns
0,0,1065-1080,1
1,0,1080-1095,1
2,0,1095-1110,1
3,0,1125-1140,1
4,0,1140-1155,2
...,...,...,...
952,29,945-960,1
953,29,960-975,3
954,29,975-990,1
955,29,990-1005,1


In [18]:
!ls ORIE-5213-Spring2023/'Bike sharing'/Dataset

Dis.json	    simu0_211.json  simu0_325.json  simu0_439.json
Initial_Inven.json  simu0_212.json  simu0_326.json  simu0_43.json
simu0_0.json	    simu0_213.json  simu0_327.json  simu0_440.json
simu0_100.json	    simu0_214.json  simu0_328.json  simu0_441.json
simu0_101.json	    simu0_215.json  simu0_329.json  simu0_442.json
simu0_102.json	    simu0_216.json  simu0_32.json   simu0_443.json
simu0_103.json	    simu0_217.json  simu0_330.json  simu0_444.json
simu0_104.json	    simu0_218.json  simu0_331.json  simu0_445.json
simu0_105.json	    simu0_219.json  simu0_332.json  simu0_446.json
simu0_106.json	    simu0_21.json   simu0_333.json  simu0_447.json
simu0_107.json	    simu0_220.json  simu0_334.json  simu0_448.json
simu0_108.json	    simu0_221.json  simu0_335.json  simu0_449.json
simu0_109.json	    simu0_222.json  simu0_336.json  simu0_44.json
simu0_10.json	    simu0_223.json  simu0_337.json  simu0_450.json
simu0_110.json	    simu0_224.json  simu0_338.json  simu0_451.json
simu0_111.json	    s

In [48]:
import os
import pandas as pd

folder_path = "ORIE-5213-Spring2023/Bike sharing/Dataset"
json_files = [file for file in os.listdir(folder_path) if (file.endswith(".json") & file.startswith("simu"))]

dataframes = []
for json_file in json_files:
    day = json_file.split("_")[1].split(".")[0]
    file_path = os.path.join(folder_path, json_file)
    df = pd.read_json(file_path)
    df["day"] = int(day)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True).sort_values(by='day')


In [49]:
combined_df

In [50]:
# def func(combined_df, combined_df_col):
#   if (combined_df_col != '0'):
#     result = combined_df[combined_df_col != 0][0] + (1440 * combined_df[combined_df_col])
#   else: 
#     result = combined_df[0]
#   return result
# combined_df['at'] = combined_df[combined_df['day'] != 0][2] + (1440 * combined_df['day']) if (combined_df['day'] != 0) else combined_df[2]

In [51]:
# new_df = combined_df.apply(lambda x: x.apply(func(x, "day")))

In [52]:
# for day in combined_df['day']:
#     if day != '0':
#         combined_df['dt'] = combined_df[0] + (1440 * int(day))
#         combined_df['at'] = combined_df[2] + (1440 * int(day))
#     else:
#         combined_df['dt'] = combined_df[0]
#         combined_df['at'] = combined_df[2]


In [ ]:
combined_df

In [35]:
combined_df['dt'] = ""
combined_df['at'] = ""

for index, row in combined_df.iterrows():
    if row['day'] != 0:
        dt = row[0] + (1440 * row['day'])
        at = row[2] + (1440 * row['day'])
    else:
        dt = row[0]
        at = row[2]
    combined_df.at[index, 'dt'] = dt
    combined_df.at[index, 'at'] = at


In [36]:
# combined_df['dt'] = ""
# combined_df['at'] = ""

# for index,row in combined_df.iterrows():
#     if row['day'] != 0:
#         dt = row[0] + (1440 * row['day'])
#         at = row[2] + (1440 * row['day'])
#         # combined_df['dt'].append(dt)
#         # combined_df['at'].append(at)
#     else:
#         dt = row[0]
#         at = row[2]
#     combined_df['dt'].append(dt)
#     combined_df['at'].append(at)


In [37]:
# def new_func(combined_df, day):
#     if day != 0:
#         combined_df['dt'] = combined_df[0] + (1440 * day)
#         combined_df['at'] = combined_df[2] + (1440 * day)
#     else:
#         combined_df['dt'] = combined_df[0]
#         combined_df['at'] = combined_df[2]

In [38]:
# new_df = combined_df.apply(lambda x: x.apply(new_func(x, combined_df["day"])))

In [41]:
pd.set_option('display.max_rows', None)

In [47]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]

,0,1,2,3,day,dt,at


In [46]:
# Adjust column 2 based on values in column 0
combined_df.loc[combined_df[0] > combined_df[2], 2] += 1440


In [ ]:
combined_df[(combined_df['day']==0) & (combined_df[2] < combined_df[0])]